# Code profiling

<font size=3> Existen varios módulos de python que ayudan a evaluar la eficiencia de los códigos. Aquí te listamos algunas muy útiles:
    
    - time : Ayuda a medir el tiempo en que se tardó en correr una función.
    - cProfile
    - pstats

In [1]:
def factorial(n):
    if n==1:
        return n
    output = n*factorial(n-1)
    return output    

In [5]:
import cProfile
import numpy as np
import planta_en_paisaje as pp

In [7]:
Temp = np.zeros([200, 400])
Borde = np.zeros([200, 400])
BordesFijos = pp.CreaBordes(Borde)
pp.plot_paisaje(Temp, BordesFijos, 0, 1.5, "hora 0")

KeyboardInterrupt: 

In [ ]:
import time
  
start = time.time()
print("Time Consumed")
print("% s seconds" % (time.time() - start))

In [17]:
cProfile.run("factorial(5)")

         8 function calls (4 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      5/1    0.000    0.000    0.000    0.000 <ipython-input-12-e5409cddbf6e>:1(factorial)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


